In [ ]:
%pip install requests beautifulsoup4 termcolor tabulate

In [ ]:
import sys

from termcolor import colored, cprint

text = colored("Hello, World!", "red", attrs=["reverse", "blink"])
print(text)
cprint("Hello, World!", "green", "on_red")

print_red_on_cyan = lambda x: cprint(x, "red", "on_cyan")
print_red_on_cyan("Hello, World!")
print_red_on_cyan("Hello, Universe!")

for i in range(10):
    cprint(i, "magenta", end=" ")

cprint("Attention!", "red", attrs=["bold"], file=sys.stderr)

# You can also specify 0-255 RGB ints via a tuple
cprint("Both foreground and background can use tuples", (100, 150, 250), (50, 60, 70))


In [ ]:
from termcolor import colored
import re
import requests
from bs4 import BeautifulSoup

my_Book_title = "The Illiad and the Odyssey"
my_Book_author = "Homer and Samuel Butler"
my_Book_rating = "4.07"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
      )
}
url = 'https://www.goodreads.com/list/show/2702.Best_Canadian_Literature'
resp = requests.get(url, headers=headers, timeout=10)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')

table = soup.find('table', class_='tableList')
rows = table.find_all('tr')[:20] # top 20 rows

titles, authors, ratings = [], [], []
for row in rows:
    t = row.find('a', class_='bookTitle')
    a = row.find('a', class_='authorName')
    m = row.find('span', class_='minirating')

# the next 3 lines use the ones in the for() function to verify the title and author are the same
# likely because the text string may vary from the one listed in the tables on the html source

    title = t.get_text(strip=True) if t else "N/A"
    author = a.get_text(strip=True) if a else "N/A"
    rating = None
    if m:
      # Extract the first decimal number (e.g., "4.23") anywhere in the text
      match = re.search(r"\d+\.\d+", m.get_text(strip=True))
      if match:
        rating = float(match.group())

      titles.append(title)
      authors.append(author)
      ratings.append(rating)

titles.append(my_Book_title)
authors.append(my_Book_author)
ratings.append(my_Book_rating)

print(colored('Books loaded!', 'green'))
for i in range(len(titles)):
    print(colored(f"{i+1}. {titles[i]} | {authors[i]} ({ratings[i]})", "green"))


file_name = "Books loaded!.txt"

with open(file_name, "w") as file:
    for i in range(len(titles)):
        file_content = f"{i+1}. {titles[i]} | {authors[i]} ({ratings[i]})\n"
        file.write(file_content)

print(colored(f"\nFile '{file_name}' has been created.", "green"))

In [ ]:
from tabulate import tabulate
textFile = open("Books loaded!.txt", "r")
contentsData = textFile.readlines()
books = contentsData
reading_idx = []
summary_authors = []
summary_counts = []

print(colored("\n ---- \t Reading List Menu \t----", "blue"))
print(colored("\nPlease view the options below", "blue"))
print(colored("\n Option 1: View Books", "blue"))
print(colored("\n Option 2: Add Book to Reading List", "blue"))
print(colored("\n Option 3: View Reading List", "blue"))
print(colored("\n Option 4: Generate Reading Summary", "blue"))
print(colored("\n Option 5: Calculate Statistics", "blue"))
print(colored("\n Option 6: Exit \n", "blue"))

while True:
  choice = input("Please select one of the options (by number):")
  
  if choice == '1':
    print(colored("----------- List of Avilable Books ----------", "green"))
    i = 0
    while i < len(books):
      print(books[i])
      i += 1


  elif choice == '2':

    book_num = int(input(f"\nEnter book number (1-{len(books)}): "))
    
    if book_num >= 1 and book_num <= len(books):
      selected_book = books[book_num - 1]
      
      reading_idx.append(selected_book)
      book_title = selected_book.split('.',1)[1]
      
      print(colored(f"\n Added to you reading list: {book_title}", "green"))

    else:
      print(colored(f"\nInvalid book number. Please enter a number between 1 and {len(books)}", "yellow"))

  elif choice == '3':
      
      if len(reading_idx) == 0:
        print(colored("\n Your Reading List is Currently Empty", "yellow"))
      
      else:
      
        print(colored("\nYour Reading list currently includes the following", "green"))
        
        i = 0
        while i < len(reading_idx):
              book_title = reading_idx[i].split('.', 1)[1]
              print(colored(f"{i + 1}. {book_title}", "green"))
              i += 1
  
  elif choice == '4':
     
    i = 0
    while i < len(reading_idx):
        book = reading_idx[i]

        
        parts = book.split("|")
        if len(parts) >= 2:
        
            author_text = parts[1]
           
            paren_index = author_text.find("(")
            if paren_index != -1:
                current_author = author_text[:paren_index].strip()
            else:
                current_author = author_text.strip()
        
        # Check if author exists
        
        author_found = False
        j = 0
        while j < len(summary_authors):
            if summary_authors[j] == current_author:
                summary_counts[j] += 1
                author_found = True
                break
            j += 1
        
        if not author_found:
            summary_authors.append(current_author)
            summary_counts.append(1)
        
        i += 1
    
    headers = ["Authors", "Number of books"]
    
    table_data = []
    i = 0
    while i < len(summary_authors):
        # Combine author and count into one row
        table_data.append([summary_authors[i], summary_counts[i]])
        i += 1
    
    print("\n")
    print(colored("----Author Summary----", "green"))
    print(colored(tabulate(table_data, headers=headers, tablefmt="grid"), "green"))
  
  elif choice == '5':
    if len(reading_idx) == 0:
        print(colored("No books selected! Cannot calculate statistical values", "yellow"))
      
    
    # Get all ratings from reading list
    else:
       print(colored("\n -------- RATING STATISTICS --------", "green"))

    ratings = []
    i = 0
    while i < len(reading_idx):
      book = reading_idx[i]
       # Find rating between parentheses
      start = book.rfind("(") + 1
      end = book.rfind(")")
     
      if start > 0 and end > start:
           
            rating_str = book[start:end]
            
            # Convert to number
            
            rating = float(rating_str)
            ratings.append(rating)
            
            i += 1
    
     # Calculate average

      total = 0
      j = 0
      while j < len(ratings):
        total += ratings[j]
        j += 1
    
      average = total / len(ratings)
      
      
      # Calculate median
      # First we start by sorting the ratings
   
      sorted_ratings = []
      k = 0
      while k < len(ratings):
        sorted_ratings.append(ratings[k])
        k += 1
    
       # Simple sorting
      m = 0
      while m < len(sorted_ratings):
        n = 0
        while n < len(sorted_ratings) - 1:
            
          if sorted_ratings[n] > sorted_ratings[n + 1]:
                # Swap
                temp = sorted_ratings[n]
                sorted_ratings[n] = sorted_ratings[n + 1]
                sorted_ratings[n + 1] = temp
          n += 1
        
        m += 1
    
    # Find middle value
      if len(sorted_ratings) % 2 == 1:
        
         median = sorted_ratings[len(sorted_ratings) // 2]
      else:
         mid1 = sorted_ratings[len(sorted_ratings) // 2 - 1]
         mid2 = sorted_ratings[len(sorted_ratings) // 2]
         median = (mid1 + mid2) / 2

      print(colored(f"Average rating: {average:.2f}", "green"))  
      print(colored(f"Median rating:  {median:.2f}", "green"))
      print(colored(f"Total books:    {len(ratings)}", "green"))
  
  elif choice == '6':
    
     print("\n")
     print(colored("\nYour Reading list has been successfully updated"))
      
     print(colored("\n -----Your list includes the following-----", "green"))
      
     i = 0
     while i < len(reading_idx):
              book_title = reading_idx[i].split('.', 1)[1]
              print("\n")
              print(colored(f"{i + 1}. {book_title}", "green"))
              i += 1
     break

  else:

    print(colored("\nInvalid choice. Please select one of the options at the top of the menu", "yellow"))

textFile.close()

                                                  Reading List Menu Logic                                                          

Display options (multiple print statements)

     Option 1: View Books

     Option 2: Add Book to Reading List

     Option 3: View Reading List

     Option 4: Generate Reading Summary

     Option 5: Calculate Statistics

     Option 6: Exit 


reading_list = reading_idx(in code)

Number chosen outside this range = "Invalid choice. Please select one of the options at the top of the menu"

while loop ---> contains all sub-functions (choice 1-6) that perform the action listed at the start of the program

               "Please select one of the options (by number):"[int from 1 to 6 select by user] = choice 

if "choice" = 1

>Print the entie list of books available (1-21) --> RETURN TO "choice"

=======================================================================================================================================

if "choice" = 2

> User selects a book to add (valid input = int between 1 & 21)

>> Valid number = book chosen is added to "reading list" --> RETURN TO "choice"

>> Invalid number = Print invalid message --> RETURN TO "choice"
 
=======================================================================================================================================
 
if "choice" = 3

> Print current "reading list" --> RETURN TO "choice"


=======================================================================================================================================


if "choice" = 4

> Extract author name from "reading list"

>> Parse through text to divide each line using parts = book.split("|") and select the second part author_text = parts[1] 

>>> Use "author_text.find" to locate "(" 

>>> Use author_text[:character from find] to define current_author

>>>> Determine if author name is already in "summary_authors"

>>>> Separate functions to iterate through summary_authors and set conditions

>>>> Determine if author name (current_author) exists in "summary_authors" list 

>>>> True = Add 1 additional count to author_count corresponding to the author

>>>> False = Append summary_authors with current_author and add another row = 1

>> table_data.append([summary_authors[i], summary_counts[i]]) = merge summary_authors & author_counts

>>> print(tabulate(table_data, headers=headers, tablefmt="grid")) = produces tabular data of "Reading Summary"

---> RETURN TO "choice"

=======================================================================================================================================

if "choice" = 5

Generate a new list = "ratings"

> Extract all rating string values from reading_list and append into "ratings" list

>> convert string to float number, add all values using a while() loop and set as "total"

>> program calculates the average = total/len(ratings)

Calculate the median 

> Re-sort the "ratings" list into "sorted_list"

*nested iteration*

>> Outer loop (m = 0) --> occurs for each element in the list & places the largest values to the very end

>>> Inner loop (n = 0) --> compares each item with every other value in the list and swaps them if they're in the wrong position

>>>> sorted_ratings[n] = sorted_ratings[n + 1] 
     sorted_ratings[n + 1] = temp 
     sorted_ratings[n + 1] = temp --> swaps position

> Calculate the median of "sorted_list" 

>> if len(sorted_ratings) % 2 == 1 --> determines if "sorted_list" has an odd number of items

>>> True --> median = sorted_ratings[len(sorted_ratings) // 2]

>>> False (even)--> mid1 = sorted_ratings[len(sorted_ratings) // 2 - 1]  # 4//2 - 1 = 2-1 = 1
                    mid2 = sorted_ratings[len(sorted_ratings) // 2]      # 4//2 = 2
                    median = (mid1 + mid2) / 2

> print(average, median, total books) ----> RETURN TO "choice"


=======================================================================================================================================


if "choice" = 6

> EXIT menu "break"
> print(closing texts) & print(reading_list)

                                                 Reflections on Work                                                                   

Major Challenges faced during construction of program code:

1. The first major barrier faced during this project was be able to import the necessary Python libraries and how to include my own personal book selection into the initial list of books. The solution essentially boiled down to adjusting the import line to the version of python being used and understanding where to position the append() function in a loop. Much of the code written was first implemented and tested on Google collab before switching over to a Jupyter notebook on VS code, and variations within the python versions didn't allow for the import of the libraries without adjusting the line of code.

2. The second, and most significant challenge, faced was during the writing and trial phase of the menu program. Because of the large volume of different functions and variables involved, it was difficult to come up with a draft code that would be able to execute each and every action. The solution is one that I discovered remembered for a different project, which is to divide each aspect of the program on seperate cells, figure out the syntax and composition of each aspect individually, and combine all the codes into one unified cell (with adjustments).

3. The third major noteworthy challenge was writing the code that would identify the part of the text containing the "author" section of the reading list, creating a sub-condition to check whether the author name was already included and then also finding a way to include a book counter in conjunction with the author name function. The solution (similar to 1) was to individually test each funtion and create the equality operator after the lists have been appended in the correct sequence. This also required intense research, as well as trial and error, to test the order of the syntax.

Additional Notes: Apologies for any confusion

1) I realized a little too late that there was no need to convert the list of books collected from Canadien reads into a text file, but that is the route I took anyway.

2) There is a bug with the ratings statistics aspect of the code that causes it to read the list multiple times that I did not have time to correct. Use the very last set of numbers.

3) theres another bug when I open the file in google collab, it repeates every output for each cell twice. I'm not sure why thats the case.

Thank you for understanding